In [2]:
import sys
sys.path.extend(['/Users/philipmassey/PycharmProjects/stock_market'])

import portfolio as pf
import market_data as md
import pandas as pd

import pymongo
from pymongo import MongoClient
client = MongoClient()
db = client['stock_market']
db.list_collection_names()

['market_data_close', 'market_data_volume']

In [3]:
from datetime import datetime
import numpy as np
import trading_calendars as tc
xnys = tc.get_calendar("XNYS")

def getStockPickleNBDays(ndays,skip=False):
    datestr = getNBusDateFromNdays(ndays,skip)
    if datestr == 'SKIP':
        return pd.DataFrame({})
    else:
        return md.getStockPickle(datestr)

def getNBusDateFromNdays(ndays,skip):
    if ndays == 0:
        dtnow = '{:%Y-%m-%d}'.format(datetime.now())
        dt = np.busday_offset(dtnow, 0, roll='backward')
        if not xnys.is_session(dt):
            return getNBusDateFromNdays(ndays + 1,skip)
    else:
        now = '{:%Y-%m-%d}'.format(datetime.now())
        dtnow = np.busday_offset(now,0,roll='backward')
        dt = np.busday_offset(dtnow,-ndays,roll='backward')
        if not xnys.is_session(dt):
            if skip:
                dt = str('SKIP')
            else:
                return getNBusDateFromNdays(ndays + 1,skip)
        #print('ndays {:d} bdate {}'.format(ndays,dt))
    return str(dt)

def printDate(ndays):
    dfall = getStockPickleNBDays(ndays,skip=True)
    if dfall.size >0:
        df = dfall['Close']
        print(ndays,str(df.index))

[printDate(ndays) for ndays in range(0,10)]


0 DatetimeIndex(['2021-01-15'], dtype='datetime64[ns]', name='Date', freq=None)
1 DatetimeIndex(['2021-01-14'], dtype='datetime64[ns]', name='Date', freq=None)
2 DatetimeIndex(['2021-01-13'], dtype='datetime64[ns]', name='Date', freq=None)
3 DatetimeIndex(['2021-01-12'], dtype='datetime64[ns]', name='Date', freq=None)
4 DatetimeIndex(['2021-01-11'], dtype='datetime64[ns]', name='Date', freq=None)
5 DatetimeIndex(['2021-01-08'], dtype='datetime64[ns]', name='Date', freq=None)
6 DatetimeIndex(['2021-01-07'], dtype='datetime64[ns]', name='Date', freq=None)
7 DatetimeIndex(['2021-01-06'], dtype='datetime64[ns]', name='Date', freq=None)
8 DatetimeIndex(['2021-01-05'], dtype='datetime64[ns]', name='Date', freq=None)
9 DatetimeIndex(['2021-01-04'], dtype='datetime64[ns]', name='Date', freq=None)


[None, None, None, None, None, None, None, None, None, None]

In [4]:
from datetime import datetime
import numpy as np
import trading_calendars as tc
xnys = tc.get_calendar("XNYS")
ndays = 7
now = '{:%Y-%m-%d}'.format(datetime.now())
dtnow = np.busday_offset(now,0,roll='backward')
dt = np.busday_offset(dtnow,-ndays,roll='backward')
if not xnys.is_session(dt):
    print('getNBusDateFromNdays')
    dt = md.getNBusDateFromNdays(ndays + 2)
print('ndays {:d} bdate {}'.format(ndays,dt))
print(str(dt))


ndays 7 bdate 2021-01-06
2021-01-06


In [17]:
def getBusDaysAgo(ndays):
    now = '{:%Y-%m-%d}'.format(datetime.now())
    dtnow = np.busday_offset(now,0,roll='backward')
    dt = np.busday_offset(dtnow,-ndays,roll='backward')
    if not xnys.is_session(dt):
        return getBusDaysAgo(ndays + 1)
    return dt
ndays=270
getBusDaysAgo(ndays)

numpy.datetime64('2020-01-03')

In [ ]:

def testbackforth(ndays):
    datestr = md.getNBusDateFromNdays(ndays,skip=True)
    ndays = md.getNBusDaysFromDateStr(datestr)
    print('{} - {} - {}'.format(ndays,datestr,ndays))

[testbackforth(ndays) for ndays in range(6,10)]

In [ ]:
mport market_data as md


def testndays(ndays):
    print('ndays ',ndays)
    start,end = md.getNDateAndToday(ndays)
    print(start,end)

def testrowndays(ndays):
    df = md.getRowNDaysAgo(ndays)
    print(df)


ndays=0
testndays(ndays)
testrowndays(ndays)
